# BERT
Use per-trained BERT Model

## Import Packages and Environment Variables

In [ ]:
import collections
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
# import torchtext
import tqdm
import transformers
from sklearn.model_selection import train_test_split
import os

In [ ]:
DATA_PATH = "data/"
DIR_PATH = os.getcwd()
TRAIN_DATA_PATH = os.path.join(DIR_PATH, DATA_PATH, "train.csv")
TEST_DATA_PATH = os.path.join(DIR_PATH, DATA_PATH, "test.csv")
VALIDATION_RATIO = 0.2
REMOVE_DUPLICATE = True
SEED = 1234

TOCKENIZER_NAME = 'bert-base-uncased'

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Data Preprocessed and Feature Engineering

In [ ]:
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)
train.head()

,review,sentiment
0,the single worst film i've ever seen in a thea...,negative
1,I was actually around 13 years old camping nea...,positive
2,A small town is attacked by a horde of bloodth...,negative
3,I think the problem with this show not getting...,positive
4,"Wow, this movie was horrible. As a Bills fan I...",negative


In [ ]:
# Remove Duplicated Data
if REMOVE_DUPLICATE:
    print(train.duplicated(subset=["review"]).sum())
    train.drop_duplicates(subset=["review"], inplace=True)
    train = train.reset_index(drop=True)

328


In [ ]:
# Tockenizer
tokenizer = transformers.BertTokenizer.from_pretrained(TOCKENIZER_NAME)

def tokenize_function(examples, tokenizer):
    return {
        "input_ids": tokenizer(examples["review"].to_list(), truncation=True)["input_ids"]
    }

In [ ]:
type(train["review"])

pandas.core.series.Series

In [ ]:
train = train.map(tokenize_function, tokenizer=tokenizer)
test = test.map(tokenize_function, tokenizer=tokenizer)

TypeError: string indices must be integers